In [1]:
import sys
sys.path.append("/home/mount/project")

In [2]:
from config_sing import *

In [102]:
import gzip
import numpy     as np
import pandas    as pd
import itertools as it
import os
from glob import glob

In [77]:
FD_WORK = "/home/mount/work"
FD_RES  = "/home/mount/work/out/proj_combeffect"

## get motifs

In [110]:
TARGET = "target_PER1"
sam    = "Input1_20x"
fdiry  = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname  = "*_merge.bed.gz"
fpath  = os.path.join(fdiry, fname)
MOTIFS = glob(fpath)
MOTIFS[:6]

['/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/HD_3_merge.bed.gz',
 '/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/MECP2_merge.bed.gz',
 '/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/FOX_1_merge.bed.gz',
 '/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/CUX_3_merge.bed.gz',
 '/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/NR_18_merge.bed.gz',
 '/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/FOX_5_merge.bed.gz']

## import one motif annotation

In [112]:
#fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_1_merge.bed.gz"

TARGET = "target_PER1"
motif  = "AP1_1_merge.bed.gz"
#motif  = "ARI5A_merge.bed.gz"

mtf = motif.replace("_merge.bed.gz", "")
sam = "Input1_20x"

fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname = motif
fpath = os.path.join(fdiry, fname)

lst = []
with gzip.open(fpath, 'rb') as file:
    for idx, line in enumerate(file):
        line = str(line, 'utf-8')
        line = line.strip().split("\t")
        
        idx1, idx2, idx3 = 5, 6, 9
        len_mtf = int(line[idx2]) - int(line[idx1])
        len_lap = int(line[idx3])
        
        if (len_mtf == len_lap):
            lst.append(line)

lst_mtf = lst
print(mtf, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True)

AP1_1 Filter 2223-2197


## create dataframe

In [113]:
CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
          "Chrom_MTF",  "Start_MTF",  "End_MTF",
          "Motif",      "Score",      "Overlap"]
CTYPES = [np.str, np.int,   np.int, np.int,
          np.str, np.int,   np.int,
          np.str, np.float, np.int]

lst = lst_mtf
dat = pd.DataFrame(lst, columns=CNAMES)
dat = dat.astype(dict(zip(CNAMES, CTYPES)))
dat = dat.assign(Sample = sam)
dat.head()

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
0,chr17,8148003,8148983,3,chr17,8148425,8148433,AP1/1,6.7321,8,Input1_20x
1,chr17,8148003,8148983,3,chr17,8148917,8148925,AP1/1,8.0514,8,Input1_20x
2,chr17,8148004,8148925,1,chr17,8148425,8148433,AP1/1,6.7321,8,Input1_20x
3,chr17,8148004,8148925,1,chr17,8148917,8148925,AP1/1,8.0514,8,Input1_20x
4,chr17,8148004,8148962,1,chr17,8148425,8148433,AP1/1,6.7321,8,Input1_20x


## get counts

In [114]:
### get #Motifs for each fragment
dat = dat  \
    .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif", "Sample"]) \
    .size() \
    .reset_index(name='N_Motif')

### get counts
dat = dat \
    .groupby(["Sample", "Motif", "N_Motif"]) \
    .agg(Value=('Count_Frag', sum)) \
    .reset_index()

dat.head()

,Sample,Motif,N_Motif,Value
0,Input1_20x,AP1/1,1,1800
1,Input1_20x,AP1/1,2,700
2,Input1_20x,AP1/1,3,1


## Exception

In [196]:
#fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_1_merge.bed.gz"

TARGET = "target_PER1"
#motif  = "AP1_1_merge.bed.gz"
motif  = "ARI5A_merge.bed.gz"

mtf = motif.replace("_merge.bed.gz", "")
sam = "Input1_20x"

fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname = motif
fpath = os.path.join(fdiry, fname)

lst = []
with gzip.open(fpath, 'rb') as file:
    for idx, line in enumerate(file):
        line = str(line, 'utf-8')
        line = line.strip().split("\t")
        
        idx1, idx2, idx3 = 5, 6, 9
        len_mtf = int(line[idx2]) - int(line[idx1])
        len_lap = int(line[idx3])
        
        if (len_mtf == len_lap):
            lst.append(line)

lst_mtf = lst
print(mtf, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True)

ARI5A Filter 767-0


In [197]:
lst_mtf

[]

## try out the loop

In [4]:
from functools import reduce

In [5]:
SAMPLES = np.concatenate([
    reduce(np.char.add, ["Input", np.arange(1,6).astype(str)        ]),
    reduce(np.char.add, ["Input", np.arange(1,6).astype(str), "_20x"]),
    reduce(np.char.add, ["TFX",   np.arange(2,6).astype(str), "_DMSO"]),
    reduce(np.char.add, ["TFX",   np.arange(2,6).astype(str), "_Dex"])
])
SAMPLES

array(['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input1_20x',
       'Input2_20x', 'Input3_20x', 'Input4_20x', 'Input5_20x',
       'TFX2_DMSO', 'TFX3_DMSO', 'TFX4_DMSO', 'TFX5_DMSO', 'TFX2_Dex',
       'TFX3_Dex', 'TFX4_Dex', 'TFX5_Dex'], dtype='<U30')

In [6]:
TARGET = "target_PER1"
sam    = "Input1_20x"
fdiry  = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname  = "*_merge.bed.gz"
fpath  = os.path.join(fdiry, fname)
MOTIFS = np.sort([os.path.basename(fp) for fp in glob(fpath)])
MOTIFS[:3]

array(['AHR_merge.bed.gz', 'AIRE_merge.bed.gz', 'AP1_1_merge.bed.gz'],
      dtype='<U26')

In [7]:
True & False

False

In [8]:
TARGET = "target_PER1"
THRESHOLD_MOTIF = 0

for motif in MOTIFS[:1]:
    
    ###
    mtf = motif.replace("_merge.bed.gz", "")
    
    for sam in SAMPLES:    

        ### set input file path
        fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
        fname = motif
        fpath = os.path.join(fdiry, fname)
        print(mtf, sam, "Import", fpath, flush=True)
        
        ### import data
        lst = []
        with gzip.open(fpath, 'rb') as file:
            for idx, line in enumerate(file):
                
                ### preprocess each line 
                line = str(line, 'utf-8')
                line = line.strip().split("\t")
                
                ### extract needed values
                idx1, idx2, idx3, idx4 = 5, 6, 8, 9
                len_mtf = int(line[idx2]) - int(line[idx1])
                len_lap = int(line[idx4])
                score   = float(line[idx3])
                
                ### filtering motifs scores and make sure
                ### fragments cover the full motif
                if (len_mtf <= len_lap) & (score >= THRESHOLD_MOTIF):
                    lst.append(line)

        print(mtf, sam, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True) 
        
        ### HANDLE EXCEPTION
        if len(lst) == 0:
            break
            
        ###
        CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
                  "Chrom_MTF",  "Start_MTF",  "End_MTF",
                  "Motif",      "Score",      "Overlap"]
        CTYPES = [np.str, np.int,   np.int, np.int,
                  np.str, np.int,   np.int,
                  np.str, np.float, np.int]
        dat = pd.DataFrame(lst, columns=CNAMES) \
            .astype(dict(zip(CNAMES, CTYPES))) \
            .assign(Sample = sam)
        
        ###
        dat = dat  \
            .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif", "Sample"]) \
            .size() \
            .reset_index(name='N_Motif')
        
        ###
        dat = dat \
            .groupby(["Sample", "Motif", "N_Motif"]) \
            .agg(Value=('Count_Frag', sum)) \
            .reset_index()
        
        print(mtf, sam, "Count", np.sum(dat.Value), flush=True) 


AHR Input1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AHR_merge.bed.gz
AHR Input1 Filter 76-76
AHR Input1 Count 44
AHR Input2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AHR_merge.bed.gz
AHR Input2 Filter 86-86
AHR Input2 Count 50
AHR Input3 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AHR_merge.bed.gz
AHR Input3 Filter 130-130
AHR Input3 Count 81
AHR Input4 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AHR_merge.bed.gz
AHR Input4 Filter 90-89
AHR Input4 Count 53
AHR Input5 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1/AHR_merge.bed.gz
AHR Input5 Filter 58-57
AHR Input5 Count 35
AHR Input1_20x Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AHR_merge.bed.gz
AHR Input1_20x Filter 1453-1450
AHR Input1_20x Count 1304
AHR Input2_20x Import /home/mount/work/out/pro

In [11]:
fdiry = os.path.join(FD_RES, 'model_linear', 'example', TARGET)
fname = "count_" + mtf + ".tsv"
fpath = os.path.join(fdiry, fname)
fpath

'/home/mount/work/out/proj_combeffect/model_linear/example/target_PER1/count_AHR.tsv'

In [17]:
#os.makedirs(fdiry, exist_ok=True)

In [14]:
#dat.to_csv(fpath, sep='\t', index=False, mode='w')

In [19]:
#dat.to_csv(fpath, sep='\t', index=False, mode='a', header=False)

## Parallelization

In [170]:
import multiprocessing
print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  56


In [228]:
def get_count_table(motif, verbose = True):
    """import annotated fragment and generate count table"""
    ### INIT
    mtf = motif.replace("_merge.bed.gz", "")
    lst_tmp = []
    
    ### for each sample, import annotated fragment and generate count table
    for sam in SAMPLES:    

        ### set input file path
        fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
        fname = motif
        fpath = os.path.join(fdiry, fname)
        if verbose:
            print(mtf, sam, "Import", fpath, flush=True)
        
        ### INIT
        lst = []
        idx = -1 # HANDLE EXCEPTION: Empty file
        
        ### import data
        with gzip.open(fpath, 'rb') as file:
            for idx, line in enumerate(file):
                
                ### preprocess each line 
                line = str(line, 'utf-8')
                line = line.strip().split("\t")
                
                ### extract needed values
                idx1, idx2, idx3, idx4 = 5, 6, 8, 9
                len_mtf = int(line[idx2]) - int(line[idx1])
                len_lap = int(line[idx4])
                score   = float(line[idx3])
                
                ### filtering motifs scores and make sure
                ### fragments cover the full motif
                if (len_mtf <= len_lap) & (score >= THRESHOLD_MOTIF):
                    lst.append(line)
        
        if verbose:
            print(mtf, sam, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True) 
            
        ### HANDLE EXCEPTION: Empty data
        if len(lst) == 0:
            if verbose:
                print(mtf, sam, "Skip Empty", flush=True) 
            continue
            
        ### wrap up a list of lines into a dataframe
        CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
                  "Chrom_MTF",  "Start_MTF",  "End_MTF",
                  "Motif",      "Score",      "Overlap"]
        CTYPES = [np.str, np.int,   np.int, np.int,
                  np.str, np.int,   np.int,
                  np.str, np.float, np.int]
        dat = pd.DataFrame(lst, columns=CNAMES) \
            .astype(dict(zip(CNAMES, CTYPES))) \
            .assign(Sample = sam)
        
        ### count the number of motifs for each fragment
        dat = dat  \
            .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif", "Sample"]) \
            .size() \
            .reset_index(name='N_Motif')
        
        ### Summarize into count table
        dat = dat \
            .groupby(["Sample", "Motif", "N_Motif"]) \
            .agg(Value=('Count_Frag', sum)) \
            .reset_index()
        
        if verbose:
            print(mtf, sam, "Count", np.sum(dat.Value), flush=True) 
        
        ### Store/Return
        lst_tmp.append(dat)
    
    if len(lst_tmp) > 0:
        return pd.concat(lst_tmp)

In [215]:
res = get_count_table(MOTIFS[0])
res

AHR Input1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AHR_merge.bed.gz
AHR Input1 Filter 76-76
AHR Input1 Count 44
AHR Input2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AHR_merge.bed.gz
AHR Input2 Filter 86-86
AHR Input2 Count 50
AHR Input3 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AHR_merge.bed.gz
AHR Input3 Filter 130-130
AHR Input3 Count 81
AHR Input4 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AHR_merge.bed.gz
AHR Input4 Filter 90-89
AHR Input4 Count 53
AHR Input5 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1/AHR_merge.bed.gz
AHR Input5 Filter 58-57
AHR Input5 Count 35
AHR Input1_20x Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AHR_merge.bed.gz
AHR Input1_20x Filter 1453-1450
AHR Input1_20x Count 1304
AHR Input2_20x Import /home/mount/work/out/pro

,Sample,Motif,N_Motif,Value
0,Input1,AHR,1,22
1,Input1,AHR,2,12
2,Input1,AHR,3,10
0,Input2,AHR,1,28
1,Input2,AHR,2,8
2,Input2,AHR,3,14
0,Input3,AHR,1,45
1,Input3,AHR,2,17
2,Input3,AHR,3,19
0,Input4,AHR,1,29


In [216]:
res = get_count_table(MOTIFS[4])
res

ARI5A Input1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/ARI5A_merge.bed.gz
ARI5A Input1 Filter 0-0
ARI5A Input1 Skip Empty
ARI5A Input2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/ARI5A_merge.bed.gz
ARI5A Input2 Filter 0-0
ARI5A Input2 Skip Empty
ARI5A Input3 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/ARI5A_merge.bed.gz
ARI5A Input3 Filter 0-0
ARI5A Input3 Skip Empty
ARI5A Input4 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/ARI5A_merge.bed.gz
ARI5A Input4 Filter 0-0
ARI5A Input4 Skip Empty
ARI5A Input5 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1/ARI5A_merge.bed.gz
ARI5A Input5 Filter 0-0
ARI5A Input5 Skip Empty
ARI5A Input1_20x Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/ARI5A_merge.bed.gz
ARI5A Input1_20x Filter 0-0
ARI5A Input1_20x Skip Empty
ARI5A 

In [222]:
pool = multiprocessing.Pool(2)
res  = pool.map(get_count_table, MOTIFS[:5])

AHR Input1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AHR_merge.bed.gz
AIRE AHR Input1 Filter Input176-76
 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AIRE_merge.bed.gz
AIRE Input1 Filter 14-13
AIRE AHRInput1  Input1 Count 44
AHR Input2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AHR_merge.bed.gz
AHR Input2 Filter 86-86
Count 12
AIRE Input2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AIRE_merge.bed.gzAHR
AIRE Input2 Filter  14-14
Input2 Count 50
AHRAIRE  Input2Input3 Count Import 11
AIRE Input3  /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AHR_merge.bed.gz
AHR Input3 Filter Import130-130
 /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AIRE_merge.bed.gz
AIREAHR Input3 Count 81
AHR Input4 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AHR_

In [224]:
MOTIFS[:5]

array(['AHR_merge.bed.gz', 'AIRE_merge.bed.gz', 'AP1_1_merge.bed.gz',
       'AP1_2_merge.bed.gz', 'ARI5A_merge.bed.gz'], dtype='<U26')

In [225]:
res[0]

,Sample,Motif,N_Motif,Value
0,Input1,AHR,1,22
1,Input1,AHR,2,12
2,Input1,AHR,3,10
0,Input2,AHR,1,28
1,Input2,AHR,2,8
2,Input2,AHR,3,14
0,Input3,AHR,1,45
1,Input3,AHR,2,17
2,Input3,AHR,3,19
0,Input4,AHR,1,29


In [226]:
res[3]

,Sample,Motif,N_Motif,Value
0,Input1,AP1/2,1,61
1,Input1,AP1/2,2,35
2,Input1,AP1/2,3,1
3,Input1,AP1/2,4,4
0,Input2,AP1/2,1,64
...,...,...,...,...
0,TFX4_Dex,AP1/2,1,1432
1,TFX4_Dex,AP1/2,2,109
2,TFX4_Dex,AP1/2,3,8
0,TFX5_Dex,AP1/2,1,1854


In [99]:
#fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_1_merge.bed.gz"

TARGET = "target_PER1"
motif  = "AP1_1_merge.bed.gz"

mtf = motif.replace("_merge.bed.gz", "")
sam = "Input1_20x"

fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname = motif
fpath = os.path.join(fdiry, fname)

lst = []
with gzip.open(fpath, 'rb') as file:
    for idx, line in enumerate(file):
        line = str(line, 'utf-8')
        line = line.strip().split("\t")
        
        idx1, idx2, idx3 = 5, 6, 9
        len_mtf = int(line[idx2]) - int(line[idx1])
        len_lap = int(line[idx3])
        
        if (len_mtf == len_lap):
            lst.append(line)
        
lst_mtf1 = lst
print(mtf, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True)

AP1_1 Filter 2223-2197


In [80]:
fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_2_merge.bed.gz"
lst = []
sam = "Input1_20x"
with gzip.open(fpath, 'rb') as file:
    for idx, line in enumerate(file):
        line = str(line, 'utf-8')
        line = line.strip().split("\t")
        
        idx1, idx2, idx3 = 5, 6, 9
        len_mtf = int(line[idx2]) - int(line[idx1])
        len_lap = int(line[idx3])
        
        if (len_mtf == len_lap):
            lst.append(line)
        
lst_mtf2 = lst

In [100]:
#fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_1_merge.bed.gz"

TARGET = "target_PER1"
motif  = "ARI5A_merge.bed.gz"

mtf = motif.replace("_merge.bed.gz", "")
sam = "Input1_20x"

fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname = motif
fpath = os.path.join(fdiry, fname)

lst = []
with gzip.open(fpath, 'rb') as file:
    for idx, line in enumerate(file):
        line = str(line, 'utf-8')
        line = line.strip().split("\t")
        
        idx1, idx2, idx3 = 5, 6, 9
        len_mtf = int(line[idx2]) - int(line[idx1])
        len_lap = int(line[idx3])
        
        if (len_mtf == len_lap):
            lst.append(line)

print(mtf, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True)

ARI5A Filter 2223-0


In [81]:
CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
          "Chrom_MTF",  "Start_MTF",  "End_MTF",
          "Motif",      "Score",      "Overlap"]
CTYPES = [np.str, np.int,   np.int, np.int,
          np.str, np.int,   np.int,
          np.str, np.float, np.int]

lst = lst_mtf1
dat = pd.DataFrame(lst, columns=CNAMES)
dat = dat.astype(dict(zip(CNAMES, CTYPES)))
dat = dat.assign(Sample = sam)
df1 = dat

lst = lst_mtf2
dat = pd.DataFrame(lst, columns=CNAMES)
dat = dat.astype(dict(zip(CNAMES, CTYPES)))
dat = dat.assign(Sample = sam)
df2 = dat

In [82]:
df1.head(3)

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
0,chr17,8148003,8148983,3,chr17,8148425,8148433,AP1/1,6.7321,8,Input1_20x
1,chr17,8148003,8148983,3,chr17,8148917,8148925,AP1/1,8.0514,8,Input1_20x
2,chr17,8148004,8148925,1,chr17,8148425,8148433,AP1/1,6.7321,8,Input1_20x


In [83]:
df2.head(3)

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
0,chr17,8148003,8148983,3,chr17,8148257,8148268,AP1/2,8.9298,11,Input1_20x
1,chr17,8148004,8148925,1,chr17,8148257,8148268,AP1/2,8.9298,11,Input1_20x
2,chr17,8148004,8148962,1,chr17,8148257,8148268,AP1/2,8.9298,11,Input1_20x


In [ ]:
dat1 = df1  \
    .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif"]) \
    .size() \
    .reset_index(name='N_Motif')

In [69]:
dat1 = df1  \
    .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif"]) \
    .agg(Value=('Count_Frag', sum))
dat1.head()

Value
Chrom_Frag Start_Frag End_Frag Count_Frag Motif       
chr17      8148003    8148983  3          AP1/1      6
           8148004    8148925  1          AP1/1      2
                      8148962  1          AP1/1      2
                      8148963  1          AP1/1      2
           8148005    8149014  1          AP1/1      2

In [85]:
dat = df1  \
    .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif", "Sample"]) \
    .size() \
    .reset_index(name='N_Motif')

dat = dat \
    .groupby(["Sample", "Motif", "N_Motif"]) \
    .agg(Value=('Count_Frag', sum)) \
    .reset_index()

dat.head()

,Sample,Motif,N_Motif,Value
0,Input1_20x,AP1/1,1,1800
1,Input1_20x,AP1/1,2,700
2,Input1_20x,AP1/1,3,1


In [86]:
dat = df2  \
    .groupby(["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif", "Sample"]) \
    .size() \
    .reset_index(name='N_Motif')

dat = dat \
    .groupby(["Sample", "Motif", "N_Motif"]) \
    .agg(Value=('Count_Frag', sum)) \
    .reset_index()

dat.head()

,Sample,Motif,N_Motif,Value
0,Input1_20x,AP1/2,1,1573
1,Input1_20x,AP1/2,2,949
2,Input1_20x,AP1/2,3,58
3,Input1_20x,AP1/2,4,59
4,Input1_20x,AP1/2,5,7


In [ ]:
df1.groupby(["Motif"]).size().reset_index(name='N_Motif')

In [46]:
dat2 = df2.loc[:,["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Motif"]]
dat2.head()

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif
0,chr17,8148003,8148983,3,AP1/2
1,chr17,8148004,8148925,1,AP1/2
2,chr17,8148004,8148962,1,AP1/2
3,chr17,8148004,8148963,1,AP1/2
4,chr17,8148005,8149014,1,AP1/2


In [49]:
dat3 = pd.merge(dat1, dat2, on=['Chrom_Frag', 'Start_Frag', "End_Frag", "Count_Frag"], how='outer')
dat3.head()

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif_x,Motif_y
0,chr17,8148003,8148983,3,AP1/1,AP1/2
1,chr17,8148003,8148983,3,AP1/1,AP1/2
2,chr17,8148004,8148925,1,AP1/1,AP1/2
3,chr17,8148004,8148925,1,AP1/1,AP1/2
4,chr17,8148004,8148962,1,AP1/1,AP1/2


In [42]:
dat1 = df1.loc[:,["Chrom_Frag"]]

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap
0,chr17,8148003,8148983,3,chr17,8148425,8148433,AP1/1,6.7321,8
1,chr17,8148003,8148983,3,chr17,8148917,8148925,AP1/1,8.0514,8
2,chr17,8148004,8148925,1,chr17,8148425,8148433,AP1/1,6.7321,8
3,chr17,8148004,8148925,1,chr17,8148917,8148925,AP1/1,8.0514,8
4,chr17,8148004,8148962,1,chr17,8148425,8148433,AP1/1,6.7321,8


In [43]:
df2.head()

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap
0,chr17,8148003,8148983,3,chr17,8148257,8148268,AP1/2,8.9298,11
1,chr17,8148004,8148925,1,chr17,8148257,8148268,AP1/2,8.9298,11
2,chr17,8148004,8148962,1,chr17,8148257,8148268,AP1/2,8.9298,11
3,chr17,8148004,8148963,1,chr17,8148257,8148268,AP1/2,8.9298,11
4,chr17,8148005,8149014,1,chr17,8148257,8148268,AP1/2,8.9298,11


In [ ]:
dat

In [ ]:
with io.TextIOWrapper(io.BufferedReader(gzip.open(filename))) as file:
    for line in file:
        print line,

In [7]:
arr = np.arange(10)
print(arr)

[0 1 2 3 4 5 6 7 8 9]


In [13]:
gen = it.islice(arr, 5)
for x in gen:
    print(x)

0
1
2
3
4
